160.963

vns
 Athletes:
['aline friess' 'mélanie de jesus dos santos' 'marine boyer'
 'carolann heduit']
Total Score:
164.76299999999998


hh
 Athletes: ['morgane osyssek' 'carolann heduit' 'mélanie de jesus dos santos'
 'lorette charpy']
Best Score: 163.629

ts
 Athletes:
['aline friess' 'marine boyer' 'mélanie de jesus dos santos'
 'carolann heduit']
Total Score:
164.929

bf
 Athletes:
['aline friess', 'carolann heduit', 'morgane osyssek', 'mélanie de jesus dos santos']
Maximum Score:
163.62900000000002

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')


In [8]:
df=df[df['Nation']=='FRA']

In [9]:
df = df.drop(columns=['Nation', 'round', 'year', 'AA'])

In [10]:
df.duplicated().sum()

5

In [11]:
df = df.drop_duplicates()

In [12]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values
})


In [9]:
df = df_max_scores.copy()

In [10]:
df

,Athlete,VT,UB,BB,FX
0,aline friess,14.333,13.233,12.266,13.100
1,carolann heduit,14.033,13.733,13.400,13.166
2,coline devillard,14.433,0.000,0.000,11.900
3,lorette charpy,0.000,14.133,12.400,0.000
4,marine boyer,13.366,12.800,13.666,13.300
5,morgane osyssek,13.333,13.100,13.366,13.433
6,mélanie de jesus dos santos,14.366,14.400,13.466,13.833


vns

In [11]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'UB', 'BB', 'VT']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['aline friess' 'mélanie de jesus dos santos' 'marine boyer'
 'carolann heduit']
Total Score:
164.76299999999998


hh

In [12]:
import pandas as pd
import numpy as np



# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'VT', 'BB', 'UB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['morgane osyssek' 'carolann heduit' 'mélanie de jesus dos santos'
 'lorette charpy']
Best Score: 163.629


ts

In [13]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score


# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 163.862
Iteration 2, Best Score: 164.929
Iteration 3, Best Score: 164.929
Iteration 4, Best Score: 164.929
Iteration 5, Best Score: 164.929
Iteration 6, Best Score: 164.929
Iteration 7, Best Score: 164.929
Iteration 8, Best Score: 164.929
Iteration 9, Best Score: 164.929
Iteration 10, Best Score: 164.929
Iteration 11, Best Score: 164.929
Iteration 12, Best Score: 164.929
Iteration 13, Best Score: 164.929
Iteration 14, Best Score: 164.929
Iteration 15, Best Score: 164.929
Iteration 16, Best Score: 164.929
Iteration 17, Best Score: 164.929
Iteration 18, Best Score: 164.929
Iteration 19, Best Score: 164.929
Iteration 20, Best Score: 164.929
Iteration 21, Best Score: 164.929
Iteration 22, Best Score: 164.929
Iteration 23, Best Score: 164.929
Iteration 24, Best Score: 164.929
Iteration 25, Best Score: 164.929
Iteration 26, Best Score: 164.929
Iteration 27, Best Score: 164.929
Iteration 28, Best Score: 164.929
Iteration 29, Best Score: 164.929
Iteration 30, Best Scor

bf

In [14]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['VT', 'UB', 'BB', 'FX']

# Generate all combinations of athletes
athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# variables to store maximum score and athletes
max_score = 0
selected_athletes = []

# tqdm progress bar
progress_bar = tqdm(total=len(athlete_combinations))

# Iterate through combination of athletes
for combination in athlete_combinations:
    # Generate all possible combinations of apparatuses for current athlete combination
    apparatus_combinations = list(itertools.permutations(apparatuses, num_athletes))

    # Iterate through each combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate the sum of top three scores for current apparatus combination
        score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    # Update progress bar
    progress_bar.update(1)

progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)


100%|███████████████████████████████████████████| 35/35 [00:02<00:00, 13.55it/s]

Selected Athletes:
['aline friess', 'carolann heduit', 'morgane osyssek', 'mélanie de jesus dos santos']
Maximum Score:
163.62900000000002


In [16]:
import pandas as pd

# Data
data = {
    'Algorithm': ['vns', 'hh', 'ts', 'bf'],
    'Athletes': [
        ['aline friess', 'mélanie de jesus dos santos', 'marine boyer', 'carolann heduit'],
        ['morgane osyssek', 'carolann heduit', 'mélanie de jesus dos santos', 'lorette charpy'],
        ['aline friess', 'marine boyer', 'mélanie de jesus dos santos', 'carolann heduit'],
        ['aline friess', 'carolann heduit', 'morgane osyssek', 'mélanie de jesus dos santos']
    ],
    'Score': [164.763, 163.629, 164.929, 163.629]
}

# Create DataFrame
fra = pd.DataFrame(data)

fra


,Algorithm,Athletes,Score
0,vns,"[aline friess, mélanie de jesus dos santos, ma...",164.763
1,hh,"[morgane osyssek, carolann heduit, mélanie de ...",163.629
2,ts,"[aline friess, marine boyer, mélanie de jesus ...",164.929
3,bf,"[aline friess, carolann heduit, morgane osysse...",163.629


In [17]:
fra['nation']='FRA'

In [18]:
fra

,Algorithm,Athletes,Score,nation
0,vns,"[aline friess, mélanie de jesus dos santos, ma...",164.763,FRA
1,hh,"[morgane osyssek, carolann heduit, mélanie de ...",163.629,FRA
2,ts,"[aline friess, marine boyer, mélanie de jesus ...",164.929,FRA
3,bf,"[aline friess, carolann heduit, morgane osysse...",163.629,FRA


In [19]:
# fra.to_csv('france.csv', index=False)

In [14]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')
df=df[df['Nation']=='FRA']
df = df.drop(columns=['Nation', 'round', 'year'])
df = df.drop_duplicates()
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})


In [15]:
df_max_scores

,Athlete,VT,UB,BB,FX,AA
0,aline friess,14.333,13.233,12.266,13.100,52.265
1,carolann heduit,14.033,13.733,13.400,13.166,53.699
2,coline devillard,14.433,0.000,0.000,11.900,26.300
3,lorette charpy,0.000,14.133,12.400,0.000,26.533
4,marine boyer,13.366,12.800,13.666,13.300,51.466
5,morgane osyssek,13.333,13.100,13.366,13.433,52.799
6,mélanie de jesus dos santos,14.366,14.400,13.466,13.833,54.465


In [16]:
df=df_max_scores.copy()

In [17]:
df.sort_values(by='AA', ascending=False)

,Athlete,VT,UB,BB,FX,AA
6,mélanie de jesus dos santos,14.366,14.400,13.466,13.833,54.465
1,carolann heduit,14.033,13.733,13.400,13.166,53.699
5,morgane osyssek,13.333,13.100,13.366,13.433,52.799
0,aline friess,14.333,13.233,12.266,13.100,52.265
4,marine boyer,13.366,12.800,13.666,13.300,51.466
3,lorette charpy,0.000,14.133,12.400,0.000,26.533
2,coline devillard,14.433,0.000,0.000,11.900,26.300


In [18]:
54.465+53.699+ 	52.799

160.963